# 0. Preliminary step: import libs, load data

In [10]:
import os
import sys
import warnings

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

import mlflow
import mlflow.pytorch

warnings.filterwarnings("ignore")
MLFLOW_SERVER_URL = 'http://127.0.0.1:5005/'

device = torch.device("cpu")

SEED=42
torch.manual_seed(SEED)


In [30]:
! echo "y" | unzip ../../assets/fashion-mnist.zip -d ../../assets/ &> /dev/null

In [11]:
train_csv = pd.read_csv("../../assets/fashion-mnist_train.csv")
test_csv = pd.read_csv("../../assets/fashion-mnist_test.csv")

In [12]:
y_train = train_csv['label'].values
X_train = train_csv.drop(['label'],axis=1).values

y_test = test_csv['label'].values
X_test = test_csv.drop(['label'],axis=1).values

BATCH_SIZE = 32

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor)
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor)

train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

# 1. Introduce teacher model to be distilled

In [22]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return X

In [23]:
def fit(model, train_loader, epoch_number=5, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    error = nn.CrossEntropyLoss()
    model.train()
    
    for epoch in range(epoch_number):
        correct = 0
        
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            if batch_idx % 200 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data, float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

def evaluate(model):
    correct = 0 
    for test_imgs, test_labels in test_loader:
        test_imgs = Variable(test_imgs).float()
        
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    score = round(float(correct) / (len(test_loader)*BATCH_SIZE), 3)
    # print("Test accuracy:{}% ".format(score))
    return score


def calc_weights(model):
    result = 0
    for layer in model.children():
        result += len(layer.weight.reshape(-1))
    return result

In [30]:
# подключаемся к серверу
mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

experiment_name = 'distill_torch_teacher'
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='/opt/mlflow_docker/mlflow_server/2', creation_time=1675984716425, experiment_id='2', last_update_time=1675984716425, lifecycle_stage='active', name='distill_torch_teacher', tags={}>

In [33]:
# подключаемся к серверу
mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

experiment_name = 'distill_torch_teacher'
mlflow.set_experiment(experiment_name)

# запуск в эксперименте
with mlflow.start_run():
    epochs = 5
    learning_rate = 1e-3

    # модель
    mlp = MLP()
    fit(mlp, train_loader, epochs, lr=learning_rate)
    
    # метрики
    n_weights = calc_weights(mlp)
    accuracy = evaluate(mlp)

    # сохраняем значения эксперимента в системе
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("n_weights", n_weights)
    mlflow.log_metric("accuracy", accuracy)
    
    mlflow.pytorch.log_model(mlp, "model")


Epoch : 0 [0/60000 (0%)]	Loss: 20.433893	 Accuracy:0.000%
Epoch : 0 [6400/60000 (11%)]	Loss: 0.715844	 Accuracy:69.745%
Epoch : 0 [12800/60000 (21%)]	Loss: 0.828387	 Accuracy:74.135%
Epoch : 0 [19200/60000 (32%)]	Loss: 0.356695	 Accuracy:75.816%
Epoch : 0 [25600/60000 (43%)]	Loss: 0.280699	 Accuracy:76.990%
Epoch : 0 [32000/60000 (53%)]	Loss: 0.561405	 Accuracy:77.700%
Epoch : 0 [38400/60000 (64%)]	Loss: 0.367472	 Accuracy:78.367%
Epoch : 0 [44800/60000 (75%)]	Loss: 0.309912	 Accuracy:78.928%
Epoch : 0 [51200/60000 (85%)]	Loss: 0.222573	 Accuracy:79.318%
Epoch : 0 [57600/60000 (96%)]	Loss: 0.184787	 Accuracy:79.555%
Epoch : 1 [0/60000 (0%)]	Loss: 0.460531	 Accuracy:87.500%
Epoch : 1 [6400/60000 (11%)]	Loss: 0.524790	 Accuracy:83.178%
Epoch : 1 [12800/60000 (21%)]	Loss: 0.705526	 Accuracy:83.611%
Epoch : 1 [19200/60000 (32%)]	Loss: 0.275794	 Accuracy:83.907%
Epoch : 1 [25600/60000 (43%)]	Loss: 0.260962	 Accuracy:84.028%
Epoch : 1 [32000/60000 (53%)]	Loss: 0.452591	 Accuracy:83.950%
Epoc

# 2. Add teacher model to registry, serve a teacher model for "why not?"

In [34]:
client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)
experiment = client.get_experiment_by_name(experiment_name)
# run_info = client.list_run_infos(experiment.experiment_id)[-1] # not working for some reason. Deprecated?
# mlflow.last_active_run().data # alternatively. ids mismatch though, yet queries correspond (strangely)
last_run_info = mlflow.search_runs([experiment.experiment_id]).iloc[0, :]

In [35]:
reg_model_name = "distill_teacher_MLP"

# региструем модель
client.create_registered_model(reg_model_name)
# создаем первую версию
result = client.create_model_version(
    name=reg_model_name,
    source=f"{last_run_info['artifact_uri']}/model",
    run_id=last_run_info['run_id']
)

print(result)

2023/02/10 02:31:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: distill_teacher_MLP, version 1


<ModelVersion: creation_timestamp=1675985472718, current_stage='None', description='', last_updated_timestamp=1675985472718, name='distill_teacher_MLP', run_id='36ee55666a9a420e9e182325256190c0', run_link='', source='/opt/mlflow_docker/mlflow_server/2/36ee55666a9a420e9e182325256190c0/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>


In [36]:
client.transition_model_version_stage(
    name=reg_model_name,
    version=1,
    stage="Staging"
)

<ModelVersion: creation_timestamp=1675985472718, current_stage='Staging', description='', last_updated_timestamp=1675985491412, name='distill_teacher_MLP', run_id='36ee55666a9a420e9e182325256190c0', run_link='', source='/opt/mlflow_docker/mlflow_server/2/36ee55666a9a420e9e182325256190c0/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [37]:
# seems like needs to be run on the mlflow_server machine. Other CLI-native options?
os.system(f'MLFLOW_TRACKING_URI=http://127.0.0.1:5005/ mlflow models serve -m "models:/{reg_model_name}/Staging" -p 5016 --no-conda &')

0

2023/02/10 02:33:04 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2023/02/10 02:33:04 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 127.0.0.1:5016 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2023-02-10 02:33:05 +0300] [67269] [INFO] Starting gunicorn 20.1.0
[2023-02-10 02:33:05 +0300] [67269] [INFO] Listening at: http://127.0.0.1:5016 (67269)
[2023-02-10 02:33:05 +0300] [67269] [INFO] Using worker: sync
[2023-02-10 02:33:05 +0300] [67272] [INFO] Booting worker with pid: 67272


## Testing connection and input format

In [105]:
import requests
import json

test = iter(test_loader)
test_imgs,  = next(test)
img0 = Variable(test_imgs).float()[0]

url = f'http://127.0.0.1:5016/invocations'

data = pd.DataFrame(img0.reshape([1, 784])).to_dict(orient='split')

http_data = json.dumps({"dataframe_split": data})

In [106]:
response = requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)

print(f'Predictions: {response.text}')

Predictions: {"predictions": [{"0": 6.712867736816406, "1": -5.011413097381592, "2": 0.6917873620986938, "3": 0.9353787302970886, "4": -0.2633971571922302, "5": -15.808575630187988, "6": 7.237793445587158, "7": -22.36628532409668, "8": -0.10114485025405884, "9": -11.855427742004395}]}


In [127]:
r = response.json()['predictions'][0]
int(max(r, key = r.get))

6

# 3. Introduce a lightweight student model

In [129]:
class StudentMLP(nn.Module):
    def __init__(self):
        super(StudentMLP, self).__init__()
        self.linear1 = nn.Linear(784,16)
        self.linear2 = nn.Linear(16,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = self.linear2(X)
        return X

In [130]:
smlp_simple = StudentMLP()
calc_weights(smlp_simple)

12704

In [133]:
def distill(teacher_model, student_model, train_loader, epoch_number=5, alpha=0.5, temperature=2, lr=1e-3):
    def error_and_output(var_X_batch, var_y_batch): # Задаем нашу особую функцию ошибки
        # Дивергенция Кульбака-Лейблера нужна, чтобы подсчитать кросс-энтропию между двумя распределениями
        # А именно между распределениями ответов модели-учителя и модели-ученика
        kldloss = nn.KLDivLoss()  
        # Для подсчета ошибки на данных воспользуемся уже готовой функцией для кросс-энтропии
        celoss = nn.CrossEntropyLoss()
        
        # Считаем выходы из сети-учителя
        teacher_logits = teacher_model(var_X_batch)
        # И выходы из сети-ученика
        student_logits = student_model(var_X_batch)
        
        # Рассчитываем распределение вероятностей ответов с помощью softmax с параметром T для сети-ученика
        soft_predictions = F.log_softmax( student_logits / temperature, dim=1 )
        # И для сети-учителя
        soft_labels = F.softmax( teacher_logits / temperature, dim=1 )
        # Считаем ошибку дистиляции - кросс-энтропию между распределениями ответов моделей
        distillation_loss = kldloss(soft_predictions, soft_labels)
        
        # Считаем ошибку на данных - кросс-энтропию между распределением ответов сети-ученика и правильным ответом
        student_loss = celoss(student_logits, var_y_batch)
        
        # Складываем с весами
        return distillation_loss * alpha + student_loss * (1 - alpha), student_logits
    
    optimizer = torch.optim.Adam(student_model.parameters(), lr=lr)
    student_model.train()
    
    # Далее обучение проходит как обычно
    for epoch in range(epoch_number):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            loss, output = error_and_output(var_X_batch, var_y_batch)
            loss.backward()
            optimizer.step()

            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            if batch_idx % 200 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data, float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

In [132]:
import itertools

alpha_list = [0.2, 0.8, ]
temperature_list = [5, 10,]
epochs_list = [5,]
lr_list = [1e-3,]

for element in itertools.product(alpha_list, temperature_list, epochs_list, lr_list):
    print(element)

(0.2, 5, 5, 0.001)
(0.2, 10, 5, 0.001)
(0.8, 5, 5, 0.001)
(0.8, 10, 5, 0.001)


In [ ]:
# подключаемся к серверу
mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

experiment_name = 'distill_torch_student'
mlflow.set_experiment(experiment_name)

import itertools

alpha_list = [0.2, 0.8, ]
temperature_list = [5, 10,]
epochs_list = [5,]
lr_list = [1e-3,]

for element in itertools.product(alpha_list, temperature_list, epochs_list, lr_list):

    # запуск в эксперименте
    with mlflow.start_run():
        alpha, temperature, epochs, learning_rate = element

        # модель
        smlp = StudentMLP()
        distill(mlp, smlp, train_loader, epoch_number=epochs ,temperature=temperature, alpha=alpha, lr=learning_rate)
        
        # метрики
        n_weights = calc_weights(smlp)
        accuracy = evaluate(smlp)

        # сохраняем значения эксперимента в системе
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("n_weights", n_weights)
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("temperature", temperature)
        
        mlflow.log_metric("accuracy", accuracy)
        
        mlflow.pytorch.log_model(smlp, "model")


# 4. Find the best model, add it to production stage, then mlflow serve it  

In [135]:
client = mlflow.tracking.MlflowClient(MLFLOW_SERVER_URL)
experiment = client.get_experiment_by_name(experiment_name)

In [140]:
# run_info = client.list_run_infos(experiment.experiment_id)[-1] # not working for some reason. Deprecated?
# mlflow.last_active_run().data # alternatively. ids mismatch though, yet queries correspond (strangely)
runs_info_df = mlflow.search_runs([experiment.experiment_id])
best_run_info = runs_info_df.iloc[runs_info_df['metrics.accuracy'].idxmax()]

In [141]:
best_run_info

run_id                                            b130105f2c964c4dbe723c0ad1bbaf72
experiment_id                                                                    3
status                                                                    FINISHED
artifact_uri                     /opt/mlflow_docker/mlflow_server/3/b130105f2c9...
start_time                                        2023-02-10 00:49:18.185000+00:00
end_time                                          2023-02-10 00:49:52.877000+00:00
metrics.accuracy                                                             0.742
params.alpha                                                                   0.2
params.epochs                                                                    5
params.n_weights                                                             12704
params.temperature                                                               5
tags.mlflow.runName                                               colorful-ram-912
tags

In [142]:
reg_model_name = "distill_student_MLP"

# региструем модель
client.create_registered_model(reg_model_name)
# создаем первую версию
result = client.create_model_version(
    name=reg_model_name,
    source=f"{last_run_info['artifact_uri']}/model",
    run_id=last_run_info['run_id']
)

client.transition_model_version_stage(
    name=reg_model_name,
    version=1,
    stage="Staging"
)

2023/02/10 04:02:39 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: distill_student_MLP, version 1


<ModelVersion: creation_timestamp=1675990959258, current_stage='None', description='', last_updated_timestamp=1675990959258, name='distill_student_MLP', run_id='36ee55666a9a420e9e182325256190c0', run_link='', source='/opt/mlflow_docker/mlflow_server/2/36ee55666a9a420e9e182325256190c0/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>


<ModelVersion: creation_timestamp=1675990959258, current_stage='Staging', description='', last_updated_timestamp=1675990959276, name='distill_student_MLP', run_id='36ee55666a9a420e9e182325256190c0', run_link='', source='/opt/mlflow_docker/mlflow_server/2/36ee55666a9a420e9e182325256190c0/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [13]:
reg_model_name = "distill_student_MLP"
# seems like needs to be run on the mlflow_server machine. Other CLI-native options?
os.system(f'MLFLOW_TRACKING_URI=http://127.0.0.1:5005/ mlflow models serve -m "models:/{reg_model_name}/Staging" -p 5017 --no-conda &')

0

2023/02/10 05:06:41 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2023/02/10 05:06:41 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 127.0.0.1:5017 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2023-02-10 05:06:42 +0300] [93527] [INFO] Starting gunicorn 20.1.0
[2023-02-10 05:06:42 +0300] [93527] [INFO] Listening at: http://127.0.0.1:5017 (93527)
[2023-02-10 05:06:42 +0300] [93527] [INFO] Using worker: sync
[2023-02-10 05:06:42 +0300] [93528] [INFO] Booting worker with pid: 93528
2023/02/10 05:06:43 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 2.2.1, required: cloudpickle==2.0.0)
 - ipython (current: 8.9.0, required: ipython==8.5.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting e

## Test served small MLP model 

In [150]:
import requests
import json

url = f'http://127.0.0.1:5017/invocations'
data = pd.DataFrame(img0.reshape([1, 784])).to_dict(orient='split')
http_data = json.dumps({"dataframe_split": data})

In [151]:

response = requests.post(url=url, headers={'Content-Type': 'application/json'}, data=http_data)

print(f'Predictions: {response.text}')

Predictions: {"predictions": [{"0": 6.712867736816406, "1": -5.011413097381592, "2": 0.6917873620986938, "3": 0.9353787302970886, "4": -0.2633971571922302, "5": -15.808575630187988, "6": 7.237793445587158, "7": -22.36628532409668, "8": -0.10114485025405884, "9": -11.855427742004395}]}


In [152]:
r = response.json()['predictions'][0]
int(max(r, key = r.get))

6